# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [56]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [57]:
#exploring the data frame
adult_dt.info()

features = adult_dt.columns[adult_dt.columns != 'income']
income = adult_dt.columns[adult_dt.columns == 'income']

# Create DataFrame X with all columns except 'income'
X = adult_dt[features]
Y= adult_dt[income]

#changing the column names to snake format (my preference)
X = X.rename(columns={
    'age': 'age',
    'workclass': 'workclass',
    'fnlwgt': 'fnlwgt',
    'education': 'education',
    'education-num': 'education_num',
    'marital-status': 'marital_status',
    'occupation': 'occupation',
    'relationship': 'relationship',
    'race': 'race',
    'sex': 'sex',
    'capital-gain': 'capital_gain',
    'capital-loss': 'capital_loss',
    'hours-per-week': 'hours_per_week',
    'native-country': 'native_country'
})

# checking X and Y information
print(X.info())
print(Y.info())

#It seems that there are not missing values in X, we have 7 numeric columns and 7 categorical columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 3.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (tot

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 

Answer/ 

The random state is a parameter used in diferent functions to control the random number generation, the purpose of this is reproducibility.

The splitting function is part of the sklearn.model_selection module, and it is used to separate/split the data into two random subsets (train and test).

When we use the splitting function we set up a random state value to control the randomness involved in splitting the dataset.

+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

To be able to reproduce the same results with the same data set, this helps with reliability.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [58]:
# importing libraries
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [59]:
#Preprocessing step
# numerical features (columns)
num_cols = ['age', 
            'fnlwgt',
            'education_num',
            'capital_gain', 
            'capital_loss', 
            'hours_per_week']

# categorical features (columns)
cat_cols = ['workclass',
            'education',
            'marital_status',
            'occupation',
            'relationship',
            'race',
            'sex',
            'native_country']

# starting the pipeline
# numerical features
nums = Pipeline([
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')), # Distance-based imputation - KNNImputer
    ('scaler', RobustScaler()) # Scale features using statistict robust to outliers - RobustScaler
])

# categorical features
cat = Pipeline([
    ('imputer', SimpleImputer(strategy = 'most_frequent')), # Impute missing values with the most frequent value - SimpleImputer
    ('encoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary')) # One-hot encode categorical features -OneHotEncoder
])

preprocessing = ColumnTransformer([
    ('num_trasnforms', nums, num_cols), # Apply num_transforms to num_cols
    ('cat_trasnforms', cat, cat_cols) # Apply cat_transforms to cat_cols
])

preprocessing

ColumnTransformer(transformers=[('num_trasnforms',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['age', 'fnlwgt', 'education_num',
                                  'capital_gain', 'capital_loss',
                                  'hours_per_week']),
                                ('cat_trasnforms',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital_status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native_country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [60]:
#Model pipeline steps
model_pipeline = Pipeline([
    ('preprocessing', preprocessing), # Applying preprocessing steps
    ('classifier', RandomForestClassifier()) # RandomForestClassifier as the classifier
])

model_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_trasnforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education_num',
                                                   'capital_gain',
                                                   'capital_loss',
                                                   'hours_per_week']),
                                                 ('cat_trasnforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital_status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native_country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [61]:
#Cross-validation steps
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42) # splitig the data set into traing and testing sets, using a random state of 42

performance = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy'] # metric to evaluate performance

results = cross_validate(model_pipeline, X_train, Y_train, cv = 5, scoring = performance) # cross validation using 5 folds

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [62]:
# converting the results to a pandas data frame and sorting by negative log loss 
results = pd.DataFrame(results).assign(experiment = 1) # converting results to a pandas data fram and assingning numer 1 to this experiment
results_sorted = results.sort_values('test_neg_log_loss') # sorting results by negative log loss
results_sorted

# the best negative log loss value is the one closest to zero, in this case -0.396471

,fit_time,score_time,test_neg_log_loss,test_roc_auc,test_accuracy,test_balanced_accuracy,experiment
0,3.680397,0.136077,-0.391058,0.893733,0.843762,0.757411,1
2,3.596855,0.124477,-0.390726,0.901822,0.852975,0.767835,1
4,3.636167,0.127063,-0.378627,0.905240,0.862162,0.783970,1
3,3.636504,0.125590,-0.368409,0.904536,0.856786,0.776618,1
1,3.599381,0.125732,-0.338997,0.905767,0.851440,0.777990,1


Calculate the mean of each metric. 

In [63]:
# mean of each performance metric 
results_sorted.mean()

fit_time                  3.629861
score_time                0.127788
test_neg_log_loss        -0.373563
test_roc_auc              0.902220
test_accuracy             0.853425
test_balanced_accuracy    0.772765
experiment                1.000000
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [64]:
# Fitting the pipeline on training data
model_pipeline.fit(X_train, Y_train)

# predictions
Y_pred = model_pipeline.predict(X_test)
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Evaluate metrics on test data
results_test = {'log_loss_test': log_loss(Y_test, Y_pred_proba),
                'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
                'accuracy': accuracy_score(Y_test, Y_pred),
                'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
                }
results_test


{'log_loss_test': 0.39698951276708205,
 'roc_auc': 0.9048139191193272,
 'accuracy': 0.8596652848149854,
 'balanced_accuracy': 0.7840002591487094}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

Asnwer:

In this step we are recoding the values of the feature when is equal to >50K we assinged the value of 1, other wise it will be 0, this is convenient beacuse it helps the algorithms to process categorical data effectively, this improves model performance, and helps the integration into machine learning pipeline. 

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.